Sources: NYC Open Data, The Trust for Public Land

In [113]:
import pandas as pd, os

os.chdir(r'Y:/Springboard')

In [114]:
parks_df = pd.read_excel('NYC_ParkBoundaries.xls', index_col = 'FID')


In [115]:
parks_df = parks_df[['GISPROPNUM','BOROUGH','ACRES','SIGNNAME','TYPECATEGO']]
parks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2253 entries, 434 to 2357
Data columns (total 5 columns):
GISPROPNUM    2253 non-null object
BOROUGH       2253 non-null object
ACRES         2253 non-null float64
SIGNNAME      2253 non-null object
TYPECATEGO    2253 non-null object
dtypes: float64(1), object(4)
memory usage: 70.4+ KB


In [116]:
athletics_df = pd.read_excel('NYC_AthleticFacilities.xls', index_col = 'FID')
athletics_df.head()

,handball,lacrosse,shape_star,borough,gispropnum,objectid,descriptio,field_numb,regulation,adult_foot,...,netball,precinct,zipcode,flagfootba,volleyball,name,kickball,t_ball,accessible,dimensions
FID,,,,,,,,,,,,,,,,,,,,,
2540,Yes,No,1139.039261,B,B001,15825,NaN,02,No,No,...,No,94,11222,No,No,NaN,No,No,No,Handball
3481,Yes,No,1114.147202,B,B001,16773,NaN,01,No,No,...,No,94,11222,No,No,NaN,No,No,No,Handball
4898,No,No,4047.910461,B,B001,18273,NaN,01,No,No,...,No,94,11222,No,No,NaN,No,No,No,Full Court
4899,No,No,4064.570068,B,B001,18274,NaN,02,No,No,...,No,94,11222,No,No,NaN,No,No,No,Full Court
1997,No,No,29600.016251,B,B007,15278,NaN,03,No,No,...,No,62,11214,No,No,NaN,Yes,Yes,No,"60ft base path, 150-200ft fence"


In [117]:
# group by park ID and get the count of amenities of interest
athletics_count_df = athletics_df.groupby('gispropnum')[['handball','tennis','basketball','adult_soft', 'track_and']].apply(lambda x: x[x=='Yes'].count())
athletics_count_df.head()

,handball,tennis,basketball,adult_soft,track_and
gispropnum,,,,,
B001,2,0,2,0,0
B007,6,8,4,3,0
B008,12,0,6,2,0
B012,1,0,2,0,0
B016,4,0,3,0,0


In [118]:
# get count of play areas per park ID
play_df = pd.read_excel('NYC_PlayAreas.xls', index_col = 'FID')
play_count_df = play_df.groupby('gispropnum')['park_name'].count()
play_count_df.head()

gispropnum
B001    5
B007    7
B008    8
B012    5
B016    5
Name: park_name, dtype: int64

In [119]:
# get count of preserves per park ID
preserve_df = pd.read_excel('NYC_Preserves.xls', index_col = 'FID')
preserve_df.info()
preserve_count_df = preserve_df.groupby('park_id')['parkname'].count()
preserve_count_df.rename = 'gispropnum'
preserve_count_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 200
Data columns (total 23 columns):
fww           158 non-null object
parkname      201 non-null object
sp_reg_des    158 non-null object
avian         158 non-null object
shape_star    201 non-null float64
aquatic       158 non-null object
fs_rte        158 non-null object
forest        158 non-null object
mammal        158 non-null object
fish          158 non-null object
label         201 non-null object
shape_stle    201 non-null float64
l_rte         158 non-null object
grass_shru    158 non-null object
herpetile     158 non-null object
insect        158 non-null object
objectid      201 non-null int64
fw_sub_typ    201 non-null int64
status        150 non-null object
salt_marsh    158 non-null object
park_id       201 non-null object
acres         201 non-null float64
invertebra    158 non-null object
dtypes: float64(3), int64(2), object(18)
memory usage: 23.6+ KB


park_id
B018    2
B057    1
B073    7
B125    1
B384    1
Name: parkname, dtype: int64

In [120]:
# get count of spray showers per park ID
spray_df = pd.read_excel('NYC_SprayShowers.xls', index_col = 'FID')
spray_df.head()
spray_count_df = spray_df.groupby('gispropnum')['name'].count()
spray_count_df.head()

gispropnum
B001    1
B007    1
B012    1
B016    1
B017    1
Name: name, dtype: int64

In [121]:
# merge dataframes/series of count data with master parks df
df_list = [parks_df,athletics_count_df]
parks_features_df = pd.merge(parks_df,athletics_count_df, left_on='GISPROPNUM', right_index=True, how='left')

In [122]:
parks_features_df = parks_features_df.merge(play_count_df.to_frame(), left_on='GISPROPNUM', right_index=True, how='left')
parks_features_df = parks_features_df.merge(preserve_count_df.to_frame(), left_on='GISPROPNUM', right_index=True, how='left')
parks_features_df = parks_features_df.merge(spray_count_df.to_frame(), left_on='GISPROPNUM', right_index=True, how='left')

In [123]:
parks_features_df.rename(columns = {'park_name':'play_area_count','parkname' : 'preserve_count', 'name':'spray_count'}, inplace = True)
parks_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2253 entries, 434 to 2357
Data columns (total 13 columns):
GISPROPNUM         2253 non-null object
BOROUGH            2253 non-null object
ACRES              2253 non-null float64
SIGNNAME           2253 non-null object
TYPECATEGO         2253 non-null object
handball           769 non-null float64
tennis             769 non-null float64
basketball         769 non-null float64
adult_soft         769 non-null float64
track_and          769 non-null float64
play_area_count    902 non-null float64
preserve_count     82 non-null float64
spray_count        478 non-null float64
dtypes: float64(9), object(4)
memory usage: 211.2+ KB


In [124]:
# read in 10MW population data
tenmin_df = pd.read_excel('NYC_Park10MW_Statistics.xls')

In [125]:
parks_features_df = parks_features_df.merge(tenmin_df, left_on = 'SIGNNAME', right_on = 'TPL_P_NAME')

In [126]:
parks_features_df = parks_features_df.fillna(0)
parks_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1705 entries, 0 to 1704
Data columns (total 33 columns):
GISPROPNUM          1705 non-null object
BOROUGH             1705 non-null object
ACRES               1705 non-null float64
SIGNNAME            1705 non-null object
TYPECATEGO          1705 non-null object
handball            1705 non-null float64
tennis              1705 non-null float64
basketball          1705 non-null float64
adult_soft          1705 non-null float64
track_and           1705 non-null float64
play_area_count     1705 non-null float64
preserve_count      1705 non-null float64
spray_count         1705 non-null float64
OBJECTID            1705 non-null int64
TPL_P_NAME          1705 non-null object
FREQUENCY           1705 non-null int64
SUM_SVCAREA         1705 non-null float64
SUM_TOTPOPSVCA      1705 non-null int64
SUM_KIDSVCA         1705 non-null int64
SUM_YOUNGPROSVCA    1705 non-null int64
SUM_SENIORSVCA      1705 non-null int64
SUM_HHILOWSVCA      1705 non